## Добавление специализированной информации к LLM

Для того, чтобы добавить какую-то специфическую информацию к языковой модели, использовать до-обучение нерационально - как правило, объем добавляемой информации пренебрежимо мал по сравнению с теми датасетами, на которых модель изначально обучалась. Поэтому для добавления информации используется подход **In-Context Learning** - добавление информации в контекст модели (т.е. в промпт).

Если предметной информации немного - можно всю её запихнуть в контекст модели. Но если объем существенный - сначала отбирают релевантные запросу документы, и их уже добавляют в контекст модели. Такой подход называется **RAG - Retrieval Augmented Generation**.

Создадим бота, который будет знать что-то про наш интенсив.

Для начала получим файл с секретными ключами для доступа в облако (ключи будут доступны на время интенсива и работы над проектами):

In [1]:
!wget https://storage.yandexcloud.net/ycpub/keys/.env

--2025-09-29 10:41:21--  https://storage.yandexcloud.net/ycpub/keys/.env
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80 [application/x-www-form-urlencoded]
Saving to: '.env'

     0K                                                       100% 26,3M=0s

2025-09-29 10:41:23 (26,3 MB/s) - '.env' saved [80/80]



### Устанавливаем необходимые библиотеки

Для начала надо установить необходимые библиотеки Python, которые понадобятся нам в работе.

In [ ]:
%pip install dotenv openai

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 55.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 40.3 MB/s eta 0:00:0000:01
  Created wheel for yandex_chain: filename=yandex_chain-0.0.9-py3-none-any.whl size=9381 sha256=aa0a91128e907eaefed56d74fd8da5b79e4ae0e98d5e884ab88ff5431b8f6c2d
  Stored in directory: /tmp/xdg_cache/pip/wheels/80/a7/88/f2d8ee42bb6c5d4e1d410f716c1939862c56f1dfc9599dd0ff
Successfully built yandex_chain
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider addin

> **ВНИМАНИЕ!!!** После установки библиотек рекомендуется зайти в пункт меню **Kernel** -> **Restart Kernel**.

Теперь получаем секретные ключи для работы с облаком из ранее скачанного файла:

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

folder_id  = os.environ['folder_id']
api_key = os.environ['api_key']


### Разговорный агент

Создадим разговорного агента, как в примере из первого дня. Не смотрите на этот страшный код

In [3]:
import io
from openai import OpenAI
from IPython.display import Markdown, display

# Выбираем модель, которую хотим использовать
#model = f"gpt://{folder_id}/yandexgpt/rc"
#model = f"gpt://{folder_id}/gemma-3-27b-it/latest"
#model = f"gpt://{folder_id}/gpt-oss-120b/latest"
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

def printx(string):
    display(Markdown(string))


class Agent():

    def __init__(self,
            name,
            model,
            instruction, 
            tools = [], search_content = [], 
            response_format = None
            ):
        self.user_sessions = {}
        self.name = name
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        self.response_format = response_format
        self.vector_store = None
        if search_content:
            i=0
            self.vector_store = client.vector_stores.create(name=f'rag_store_{self.name}')
            for c in search_content:
                f = client.files.create(
                        purpose="assistants",
                        file = (f'rag_{self.name}_{i}.txt',io.BytesIO(c.encode("utf-8")),'text/markdown'))
                client.vector_stores.files.create(file_id=f.id, vector_store_id=self.vector_store.id)
                print(f" + Uploading rag_{self.name}_{i}.txt as id={f.id} to store={self.vector_store.id}")
                i+=1
            self.tools.append({
                "type" : "file_search",
                "vector_store_ids" : [self.vector_store.id],
                "max_num_results" : 5,
            })
            
    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default',return_raw=False):
        s = self.user_sessions.get(session_id,{ 'previous_response_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        txt = None
        if self.response_format:
            txt = {
                "format" : {
                    "type" : "json_schema",
                    "name" : "struct_out",
                    "schema" : self.response_format.model_json_schema()
                }
            }
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            previous_response_id = s['previous_response_id'],
            input = message,
            text = txt
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        # MCP Approval Requests
        mcp_approve = [ item for item in res.output if item.type == "mcp_approval_request"]
        if mcp_approve:
            res = client.responses.create(
                model=self.model,
                previous_response_id=res.id,
                tools = self.tools,
                input=[{
                    "type": "mcp_approval_response",
                    "approve": True,
                    "approval_request_id": m.id
                }
                for m in mcp_approve
                ])
        s['previous_response_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id] = s
        if return_raw:
            return res
        if self.response_format:
            return self.response_format.model_validate_json(res.output_text)
        else:
            return res.output_text

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

Зададим какой-то нейтральный системный промпт и попробуем спросить LLM про интенсив:

In [7]:
instruction = """
Ты - ассистент студентов в Школе Родченко. Твоя задача - отвечать на вопросы по школе и её
мероприятиям.
"""

agent = Agent('helper',model,instruction)

printx(agent("Что ты знаешь про облачный интенсив Яндекса?"))

На данный момент у меня нет информации о каком-либо «облачном интенсиве Яндекса», который проходит или планируется в рамках Школы Родченко.

Если ты имеешь в виду образовательный проект от Яндекса (например, Яндекс.Практикум, Яндекс.Лицей или другие интенсивы по программированию и технологиям), то такие программы существуют, но они не связаны напрямую со Школой Родченко, которая специализируется на дизайне, фотографии, типографике и смежных дисциплинах.

Если у тебя есть конкретный интенсив в виду — уточни, пожалуйста, название или контекст, и я постараюсь помочь точнее.

Чтобы добавить документы к нашему агенту - необходимо передать их в поле `search_content`. В нашем случае самая ценная иноформация об интенсиве содержится в папке `data`. При выполнении кода в Google Colab - скачаем файлы:

In [ ]:
!wget https://github.com/yandex-datasphere/ai_art_intensive/raw/refs/heads/main/Day-3/data.zip
!unzip data.zip

Теперь создадим ассистента: для этого скачаем все файлы из директории `data` и укажем содержимое при создании бота:

In [8]:
from glob import glob

content = []
for fn in glob("data/*.txt"):
    with open(fn,encoding='utf-8') as f:
        content.append(f.read())

agent = Agent('helper',model,instruction,search_content=content)

 + Uploading rag_helper_0.txt as id=fvtl07lcn4aagksuc82t to store=fvt7u4di3qjcsdu0u83e
 + Uploading rag_helper_1.txt as id=fvtp5l212smtrfb4930o to store=fvt7u4di3qjcsdu0u83e
 + Uploading rag_helper_2.txt as id=fvtvhvshtsnf38cpc3gg to store=fvt7u4di3qjcsdu0u83e


Немного подождём, пока информация проиндексируется, и зададим тот же вопрос:

In [9]:
printx(agent("Что ты знаешь про облачный интенсив Яндекса?"))

Облачный интенсив Яндекса — это двухнедельная образовательная программа, проводимая совместно Школой Родченко и Yandex Cloud. Она направлена на стимулирование использования облачных технологий и искусственного интеллекта в современном цифровом искусстве. Участниками становятся ученики и выпускники Школы Родченко, отобранные по результатам Open Call.

Программа включает:

- **Первая неделя**: семинары, примеры применения облачных технологий в творческих проектах, формирование команд и разработка идей.
- **Вторая неделя**: проектная работа, в ходе которой создаются прототипы художественных произведений. Лучшие проекты по решению жюри могут быть представлены на итоговой выставке в Галерее Краснохолмская.

Идеологом и ведущим программы является Дмитрий Сошников — эксперт по ИИ и облачным технологиям, ранее работавший в Microsoft. Он активно продвигает использование современных технологий в искусстве.

**Основные правила участия:**
1. Посещать все мероприятия и проявлять любопытство.
2. Не опаздывать.
3. Получать удовольствие от процесса.

In [10]:
printx(agent('Кто может участвовать в мероприятии?'))

Участвовать в облачном интенсиве Яндекса могут ученики и выпускники Школы Родченко, отобранные по результатам Open Call. Мероприятие ориентировано на тех, кто интересуется цифровым искусством, облачными технологиями и искусственным интеллектом, и готов работать в команде над созданием художественных проектов с использованием современных технологий.

## Выводы

RAG позволяет очень просто добавить информацию к языковой модели. Однако, чтобы добиться устойчивой работы RAG, необходимо:

* Поэкспериментировать с настройками бота чтобы убедиться, что поисковый инструмент вызывается когда нужно
* Использовать такую текстовую базу знаний, в которой в каждом фрагменте содержится одна законченная мысль небольшого объема. Автоматическое разбиение длинных текстов на куски (даже с перекрытием) работает не всегда хорошо
* Экспериментировать с промптами для более качественного ответа